**Important Note: About the notebook: The idea was to download torrents to GDrive using colabs. Do not use this for wrong purposes.**


---


**Note:** To get more disk space:
> Go to Runtime -> Change Runtime and give GPU as the Hardware Accelerator.
Please note that there is a limit on this disk space. Do some searchings on the maximum storage that you can get from colabs.


> ## Start Session (Run the following code segment)

You'll asked to provide a token to your GDrive. Please use the suggested link at the end to get the token. Once you got it, paste it on the space and press enter.

In [ ]:
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!apt install python3-libtorrent

import libtorrent as lt
from google.colab import drive

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []
drive.mount("/content/drive")

## Use one of the followings

### Add torrent file (give the path to your torrent file)

In [ ]:
from google.colab import files

source = files.upload()
params = {
    "save_path": "/content/drive/My Drive/Torrent",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

### Provide the magnet link

Once you run this code, there will be a blank space to provide the magnet link. You can provide multiple magnet links to a one run(input links one by one following adding a link and pressing enter). After you done with adding links type **Exit** to continue to the next step.

In [ ]:
params = {"save_path": "/content/drive/My Drive/Torrent"}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )

### Start the downloading.

The files will be save to your GDrive in the folder named as **Torrent**

In [ ]:
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)
